# Montando drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importando Pacotes

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


ratings = pd.read_csv('/content/drive/MyDrive/sistema de recomendação/ratings.csv')
items = pd.read_csv('/content/drive/MyDrive/sistema de recomendação/items.csv')

In [4]:
utility_matrix = ratings.pivot(index='userId', columns='jokeId', values='rating')


## Pré-processamento e exploração de dados

In [5]:
# Exemplo dos primeiros registros
print(ratings.head())
print(items.head())

   userId  jokeId  rating
0       1       5   0.219
1       1       7  -9.281
2       1       8  -9.281
3       1      13  -6.781
4       1      15   0.875
   jokeId                                           jokeText
0       1  A man visits the doctor. The doctor says "I ha...
1       2  This couple had an excellent relationship goin...
2       3  Q. What's 200 feet long and has 4 teeth? \n\nA...
3       4  Q. What's the difference between a man and a t...
4       5  Q.\tWhat's O. J. Simpson's Internet address? \...


In [17]:
ratings.dtypes

userId      int64
jokeId      int64
rating    float64
dtype: object

In [18]:
items.dtypes

jokeId       int64
jokeText    object
dtype: object

In [22]:
items.describe()

,jokeId
count,150.000000
mean,75.500000
std,43.445368
min,1.000000
25%,38.250000
50%,75.500000
75%,112.750000
max,150.000000


In [23]:
ratings.describe()


,userId,jokeId,rating
count,1.761439e+06,1.761439e+06,1.761439e+06
mean,3.272322e+04,7.071133e+01,1.618602e+00
std,1.828011e+04,4.600790e+01,5.302608e+00
min,1.000000e+00,5.000000e+00,-1.000000e+01
25%,1.720200e+04,2.100000e+01,-2.031000e+00
50%,3.480800e+04,6.900000e+01,2.219000e+00
75%,4.730600e+04,1.120000e+02,5.719000e+00
max,6.397800e+04,1.500000e+02,1.000000e+01


In [11]:
print(f'Shape of item_similarity: {item_similarity.shape}')


Shape of item_similarity: (140, 140)


In [12]:
print(f'Shape of utility_matrix: {utility_matrix.shape}')


Shape of utility_matrix: (59132, 140)


In [14]:
print(f'User Ratings for {user_id}: {user_ratings}')
print(f'Similar Scores for {user_id}: {similar_scores}')
print(f'Recommended Items for {user_id}: {recommended_items}')


User Ratings for 8992: jokeId
5       0.000
7       9.844
8     -10.000
13      0.000
15      0.000
        ...  
146     0.000
147     0.000
148     0.000
149     0.000
150     0.000
Name: 8992, Length: 140, dtype: float64
Similar Scores for 8992: jokeId
5      0.145394
7      6.104106
8     -6.318448
13    -0.660515
15     0.356318
         ...   
146   -0.161891
147   -0.825074
148   -0.978484
149   -0.895433
150   -0.911197
Length: 140, dtype: float64
Recommended Items for 8992: jokeId
15     0.356318
141    0.340223
124    0.296618
58     0.217941
5      0.145394
         ...   
105   -1.285030
32    -1.313405
106   -1.341069
114   -1.362312
89    -1.432572
Length: 138, dtype: float64


In [15]:
# Criar matriz de utilidade
utility_matrix = ratings.pivot(index='userId', columns='jokeId', values='rating')

# Calcular a variância das avaliações para cada usuário
user_variance = utility_matrix.var(axis=1)

# Selecionar os IDs dos 3 usuários com maior variância
top_var_users = user_variance.nlargest(3).index.tolist()

print(f"Os três usuários com gostos mais variados têm os IDs: {top_var_users}")


Os três usuários com gostos mais variados têm os IDs: [54631, 63832, 8992]


In [8]:
# Preenchendo os valores NaN com 0, porque vamos calcular a similaridade do cosseno
utility_matrix = utility_matrix.fillna(0)

# Calculando a similaridade do cosseno
item_similarity = cosine_similarity(utility_matrix.T)


In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Suponha que 'ratings' seja o seu DataFrame original
# ratings = pd.DataFrame(...)

# E que 'items' seja o DataFrame com as informações das piadas
# items = pd.DataFrame(...)

# Criar matriz de utilidade
utility_matrix = ratings.pivot(index='userId', columns='jokeId', values='rating')

# Calcular a variância das avaliações para cada usuário
user_variance = utility_matrix.var(axis=1)

# Selecionar os IDs dos 3 usuários com maior variância
top_var_users = user_variance.nlargest(3).index.tolist()

print(f"Os três usuários com gostos mais variados têm os IDs: {top_var_users}")

# Calcular a matriz de similaridade de itens usando similaridade de cosseno
item_similarity = cosine_similarity(utility_matrix.fillna(0).T)
item_similarity_df = pd.DataFrame(item_similarity, index=utility_matrix.columns, columns=utility_matrix.columns)

# Usar os IDs de usuários com a maior variância
for user_id in top_var_users:
    # Pegue as classificações para um usuário específico
    user_ratings = utility_matrix.loc[user_id].fillna(0)

    # Calcule as pontuações recomendadas
    similar_scores = item_similarity_df.dot(user_ratings).dropna()

    # Classifique os itens com base nas pontuações
    recommended_items = similar_scores.sort_values(ascending=False)

    # Filtrar os itens que o usuário já classificou
    already_rated = utility_matrix.loc[user_id].dropna().index
    recommended_items = recommended_items.drop(already_rated)

    # Pegar o top 5 de piadas que o usuário ainda não avaliou
    top_5_recommended = recommended_items.nlargest(5).index.tolist()

    # Buscar os textos das piadas no DataFrame 'items'
    top_5_jokes = items[items['jokeId'].isin(top_5_recommended)]

    print(f"Top 5 anedotas recomendadas para o usuário {user_id} são:")
    print(top_5_jokes)




Os três usuários com gostos mais variados têm os IDs: [54631, 63832, 8992]
Top 5 anedotas recomendadas para o usuário 54631 são:
     jokeId                                           jokeText
31       32  A man arrives at the gates of heaven. St. Pete...
88       89  A radio conversation of a US naval \nship with...
104     105  A couple of hunters are out in the woods in th...
105     106  An engineer dies and reports to the pearly gat...
113     114  Sherlock Holmes and Dr. Watson go on a camping...
Top 5 anedotas recomendadas para o usuário 63832 são:
     jokeId                                           jokeText
4         5  Q.\tWhat's O. J. Simpson's Internet address? \...
14       15  Q:  What did the blind person say when given s...
57       58  How many teddybears does it take to change a l...
123     124  Person 1: Hey, wanna hear a great knock-knock ...
140     141  Jack Bauer can get McDonald's breakfast after ...
Top 5 anedotas recomendadas para o usuário 8992 são:
     jok

### Justificação da Escolha dos Utilizadores

Os utilizadores foram escolhidos com base na variância das suas avaliações para as anedotas. A variância é uma medida estatística que descreve o quão diferentes são os valores em um conjunto de dados. No contexto do sistema de recomendação, uma variância alta sugere que o utilizador tem uma gama diversificada de gostos, dando altas e baixas classificações para diferentes anedotas. Esses utilizadores são particularmente interessantes para o sistema de recomendação, porque representam um desafio maior em termos de previsão de suas preferências. Além disso, ao focar nesses utilizadores, podemos otimizar o sistema para casos mais complexos, o que geralmente leva a melhorias na qualidade das recomendações para todos os utilizadores.

### Abordagem Escolhida para Recomendação de Anedotas

A abordagem escolhida para a recomendação de anedotas é baseada na similaridade de cosseno entre os itens, que são as anedotas neste caso. Esta abordagem é frequentemente referida como "filtragem colaborativa baseada em itens". A similaridade de cosseno é calculada entre cada par de anedotas com base nas avaliações dos utilizadores. Em seguida, para fazer recomendações para um utilizador específico, o sistema prevê a classificação que o utilizador daria a anedotas que ainda não classificou, usando suas classificações anteriores e as similaridades de cosseno previamente calculadas.

### Justificação da Abordagem

A razão pela qual a filtragem colaborativa baseada em itens foi escolhida é por causa de sua eficácia e simplicidade. Primeiramente, essa abordagem pode ser muito eficaz quando se tem uma matriz de utilidade esparsa, que é frequentemente o caso em sistemas de recomendação. A similaridade de cosseno tem a vantagem de levar em consideração o ângulo entre os vetores de avaliação, em vez da magnitude, tornando-a uma métrica mais robusta em diferentes escalas.

Além disso, a abordagem baseada em itens é computacionalmente mais eficiente quando se tem um grande número de utilizadores e um número relativamente estável de itens. Uma vez calculada, a matriz de similaridade de itens pode ser reutilizada para fazer recomendações para vários utilizadores, sem a necessidade de recálculos frequentes.

Em resumo, a filtragem colaborativa baseada em itens oferece um bom equilíbrio entre eficácia e eficiência, tornando-a uma escolha apropriada para este contexto.